# Report Handling Breakdown

This Dashboard gives some insight into the flow and handling of reports by section

1. The leftmost block shows the total number of reports that were received in the Portal that are now closed.
2. The middle stack of blocks show how many of these reports make it to each section.
3. Finally, the last column of blocks shows the last contact that was made to the report before it was closed.

In [ ]:
%load_ext sql
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [ ]:
%%sql contact_breakdown <<
WITH last_touch AS (
  SELECT
    actor_object_id,
    target_object_id,
    description,
    verb,
    ROW_NUMBER() OVER(PARTITION BY target_object_id ORDER BY timestamp DESC) AS row_number
  FROM actstream_action
  WHERE verb = 'Contacted complainant:'
),

contact_actions AS (
  SELECT
    actor_object_id as user_id,
    NULLIF(target_object_id, '')::int as report_id,
    CASE
      WHEN strpos(description, 'Email sent:') = 1
        THEN SUBSTR(description, STRPOS(description, '''') + 1, STRPOS(description, ''' to') - STRPOS(description, '''') - 1)
      WHEN strpos(description, 'Printed') = 1
        THEN SUBSTR(description, STRPOS(description, '''') + 1, STRPOS(description, ''' template') - STRPOS(description, '''') - 1)
      WHEN strpos(description, 'Copied') = 1
        THEN SUBSTR(description, STRPOS(description, '''') + 1, STRPOS(description, ''' template') - STRPOS(description, '''') - 1)
      ELSE
         'Other'
    END AS action
  FROM last_touch
  WHERE row_number = 1
)

SELECT
  contact_actions.action as action,
  cts_forms_report.assigned_section AS section,
  COUNT(DISTINCT contact_actions.user_id) AS user_count,
  COUNT(DISTINCT contact_actions.report_id) AS report_count
FROM contact_actions
LEFT JOIN cts_forms_report
ON contact_actions.report_id = cts_forms_report.id
GROUP BY ROLLUP (section, action)

In [ ]:
import plotly.graph_objects as go

import collections

root = next(
    row
    for row in contact_breakdown
    if row.section is None and row.action is None
)

section_roots = [
    row
    for row in contact_breakdown
    if row.section is not None and row.action is None
]

leafs = [
    row
    for row in contact_breakdown
    if row.section is not None and row.action is not None
]

indices = {
    (row.section, row.action): i
    for i, row in enumerate([
        root,
        *section_roots,
        *leafs,
    ])
}

node = {
    "pad": 15,
    "thickness": 15,
    "line": {
        "color": "black",
        "width": 0.5
    },
    "label": [
        'Intake',
        *[
            row.section
            for row in section_roots
        ],
        *[
            row.action
            for row in leafs
        ]
    ]
}

link = {
    "source": [
        *[
            indices[(root.section, root.action)]
            for _ in section_roots
        ],
        *[
            indices[(leaf.section, None)]
            for leaf in leafs
        ],
    ],
    "target": [
        *[
            indices[(section.section, None)]
            for section in section_roots
        ],
        *[
            indices[(leaf.section, leaf.action)]
            for leaf in leafs
        ],
    ],
    "value": [
        *[
            section.report_count
            for section in section_roots
        ],
        *[
            leaf.report_count
            for leaf in leafs
        ],
    ],
}

fig = go.Figure(data=[go.Sankey(
    valueformat = ".0f",
    valuesuffix = " Reports",
    node = node,
    link = link,
)])

fig.update_layout(title_text="", font_size=10, height=1500)

fig.show()